# 45-Topic Model of the Constantine Beys Corpus

## Methods



## Initial Analysis



## Experimental Visualizations

In [1]:
import pandas as pd

oa45_2_topics = pd.Cov = pd.read_csv("/Users/asg/mallet/output/OA-model/oa45_2-keys.csv", sep=',', names=["topic_number", "topic_weight", "top_words"])

oa45_2_topics.style.set_properties(subset=['top_words'], **{'width': '600px'})

,topic_number,topic_weight,top_words
0,0,0.04249,gana biskra région fondé avantage ensemble frères épousa main gendre partager répandit l’encontre mariage restituer redjas el-arab garnison tripoli beau-père
1,1,0.04283,chaouch l’usurpateur dieu joug l’agha lettre bach-agha goums traître expédia fers el-kebaïli ordres fonctionnaires avez conformer l’informer l’odjak douaer deïra
2,2,0.01828,françaises valée aïssa français azedine france territoires ministre l’émir impérialistes derradji autorités laissant octobre réponse consul droits coudiat côte propositions
3,3,0.00208,perrégaux brave lemercier démarche dernières l’état-major brigade mur ben-labiod magasin bride descendit direction goum cigognes l’obstacle dépression damré morland inouïs
4,4,2.11202,bey pacha alger d’une mort fils province qu’il d’alger dit pays eut gouvernement faire temps pouvoir jusqu’à tribus d’un fin
5,5,0.92226,ville turcs hommes troupes colonne camp grande grand puis commandement temps enfin nombre prit forces cavaliers tête place marche s’était
6,6,0.04467,discipline décapiter bacri fermeté corporation finances croire rigueur d’avoir entretenir réaliser juif chargements hadj-ali sévir frères insurgés el-djihad traversé commandements
7,7,0.02833,sidi bach-agha hassan enterré tombe hanek s’est rue travaux supérieure lakhdar feinte savait incidents bou-hanek-bey cavalier soutenu foi constructions propriété
8,8,0.34967,dey tunisie nouveau révolte alger rupture nommé medjana janissaires luttes défaite provinces d'alger refuge fonctions obtint décida fureur essaya commencer
9,9,0.14915,khalifa hadj makhzen chaouchs kaïd saïd touggourt kaïd-dar el-memlouk poste biens arrêter bach-kateb mostefa remplacé confia bach-serradj partisans ouled présentèrent


In [ ]:
# Create a list of topic labels.
labels = ['Ott suppression', '1806 Tunisian siege', 'govern*', 'battle of Con.', 'public works', 'La Calle_Tunisia', 'religion', 'grand tribes', 'internal pol appts & unrest']

# Add the labels list as a new column to the dataframe
oa9_topics['labels'] = labels

# Adjust the style of the dataframe so that the column of top_words is wide enough to see all of them.
oa9_topics.style.set_properties(subset=['top_words'], **{'width': '500px'})

In [ ]:
# Represent the weights of the topics as a simple bar chart, using the author-created topic labels. 

topic_bar = oa9_topics.plot.bar(x='labels', y='topic_weight', rot=0, figsize=(20,5))

In [ ]:
# The mallet .txt doc-topics output file was converted to a csv, and the columns for 
# document numbers and filepaths were deleted in advance of reading the csv into a 
# pandas dataframe. Be sure to convert the columns to number data type with ~6 decimal places to remove the 
# exponential format. Also, add a new line at the top of the csv with the labels defined in the previous step.

oa9_docs = pd.Cov = pd.read_csv("/Users/asg/mallet/output/OA-model/oa9-doc-topics.csv", sep=',')

# Uncomment the following line to view dataframe
#oa9_docs

In [ ]:
# Represent the topic weights in each document as a heatmap.

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams['figure.figsize'] = (17.0, 10.0)

oa9_docs = oa9_docs[oa9_docs.columns].astype(float)

# Create a heatmap that is visually accessible (see https://matplotlib.org/tutorials/colors/colormaps.html)
# using the 'inferno' colormap. 

sns.heatmap(oa9_docs, cmap='inferno', vmin=0.0, vmax=1.0)

In [ ]:
# Create a hierarchical heatmap using seaborn's "clustermap" method with default settings.
# By default this method will cluster by both rows and columns, which is what is most useful for this data.

sns.clustermap(oa9_docs, cmap='inferno')

## Visualizing Mallet output with pyLDAvis
You will need to generate MALLET's output-topic-states <b>gz</b> file to prepare the MALLET output in the format needed by pyLDAvis. I didn't know about this possibility, so I had to run MALLET again, creating a different 5-topic model than the ones visualized above. 

Huge thanks to <a href="https://jeriwieringa.com/about/">Jeri E. Wieringa</a> for making this connection and sharing all of the code! You can find her explanation and code at <a href="https://jeriwieringa.com/2018/07/17/pyLDAviz-and-Mallet/">"Using pyLDAvis with Mallet."</a>

In [2]:
import pandas as pd
import gzip
import os

dataDir = "/Users/asg/mallet/output/OA-model/"

def extract_params(statefile):
    """Extract the alpha and beta values from the statefile. 
    
    Args:
        statefile (str): Path to the statefile produced by Mallet.
    Returns:
        tuple: alpha (list), beta
    """
    with gzip.open(statefile, 'r') as state:
        params = [x.decode('utf-8').strip() for x in state.readlines()[1:3]]
    return (list(params[0].split(":")[1].split(" ")), float(params[1].split(":")[1]))

def state_to_df(statefile):
    """Transform state file into pandas dataframe. 
    The Mallet statefile is tab-separated, and the first two rows contain the alpha and beta hyperparameters.
    
    Args:
        statefile (str): Path to statefile produced by Mallet.
    Returns:
        dataframe: topic assignment for each token in each document of the model
    """
    return pd.read_csv(statefile,
                      compression='gzip',
                      sep=' ',
                      skiprows=[1,2]
                      )

In [4]:
params = extract_params(os.path.join(dataDir, 'oa9-topic-state.gz'))

alpha = [float(x) for x in params[0][1:]]
beta = params[1]
print("{}, {}".format(alpha, beta))

df = state_to_df(os.path.join(dataDir, 'oa45_2-topic-state.gz'))

[0.21643402806355239, 0.3234136774257482, 1.7569832677512662, 0.024844683515057787, 0.06930723228392457, 0.34945168610637367, 0.16325988915938713, 0.20354210248367857, 0.1662741857413399], 0.11425618862977559


In [5]:
# Define column 'type' as a string to avoid nan pandas errors.
df['type'] = df.type.astype(str)

# Preview the dataframe from the statefile, which will then be used to generate the data needed for the 
# LDAvis library. 
df[:4]

,#doc,source,pos,typeindex,type,topic
0,0,data/OttAlg/gaid-36.txt,0,0,naaman-bey,34
1,0,data/OttAlg/gaid-36.txt,1,1,mai,4
2,0,data/OttAlg/gaid-36.txt,2,2,turc,4
3,0,data/OttAlg/gaid-36.txt,3,3,naissance,4


<b>Step 1:</b> We need the length of the documents. To do this, Jeri grouped the data by document id and counted the tokens in the doc. This data is then sorted by the doc id, so it will be in the correct order for visualization pre-processing.

In [6]:
# Get document lengths from statefile.
docs = df.groupby('#doc')['type'].count().reset_index(name='doc_length')

# Preview the gathered & grouped data:
docs[:4]

,#doc,doc_length
0,0,579
1,1,2
2,2,17
3,3,320


<b>Step 2:</b> We need the voabulary and frequencies. Jeri used pandas to generate a new frame with the word counts. She then sorted this df so that it is alphabetical by type, a step that will need to be repeated when creating the topic-term matrix because the order matters and must be kept consistent. Just as in Jeri's example, I, too, have OCR errors (see the first entry in the vocab dataframe above).

In [7]:
# Get vocab and term frequencies from the statefile.
vocab = df['type'].value_counts().reset_index()
vocab.columns = ['type', 'term_freq']
vocab = vocab.sort_values(by='type', ascending=True)

# Preview the resulting dataframe
vocab[:4]

,type,term_freq
15199,a*tes,1
15050,a-t-elle,1
4904,a-t-il,3
8164,abaissement,2


<b>Step 3:</b> Create matrix files. The values need to be smoothed and the data normalized so that each row sums to 1. Jeri used sklearn to handle large matrices efficiently. 

In [8]:
# Topic-term matrix from state file.

import sklearn.preprocessing

def pivot_and_smooth(df, smooth_value, rows_variable, cols_variable, values_variable):
    """
    Turns the pandas dataframe into a data matrix.
    Args:
        df (dataframe): aggregated dataframe
        smooth_value (float): value to add to the matrix to account for the priors
        rows_variable (str): name of dataframe row to use as the rows in the matrix
        cols_variable (str): name of dataframe column to use as the columns in the matrix
        values_variable (str): name of the dataframe column to use as the values in the matrix
    Returns:
        dataframe: pandas matrix that has been normalized on the rows
    """
    matrix = df.pivot(index=rows_variable, columns=cols_variable, values=values_variable).fillna(value=0)
    matrix = matrix.values + smooth_value
    normed = sklearn.preprocessing.normalize(matrix, norm='l1', axis=1)
    return pd.DataFrame(normed)

<b>Step 4:</b> [From Jeri's post] Next, we need to aggregate the data from the statefile dataframe to get the nmber of topic assignments for words in the documents. For <b>phi</b>, the topic-term matrix, [she] aggregated by topic and word, counted the number of times each word was assigned to each topic, and then sorted the resulting dataframe alphabetically by word, so that it matches the order of the vocabulary frame. Here, [she] used the <b>beta</b> hyperparameter as the smoothing value.

In [9]:
phi_df = df.groupby(['topic', 'type'])['type'].count().reset_index(name ='token_count')
phi_df = phi_df.sort_values(by='type', ascending=True)

phi_df[:10]

phi = pivot_and_smooth(phi_df, beta, 'topic', 'type', 'token_count')

phi[:10]

,0,1,2,3,4,5,6,7,8,9,...,16790,16791,16792,16793,16794,16795,16796,16797,16798,16799
0,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,...,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044,0.000044
1,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,...,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038
2,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,...,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042
3,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,...,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047
4,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000400,0.000006,0.000006,...,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
5,0.000008,0.000008,0.000227,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
6,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,...,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048
7,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,...,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043
8,0.000028,0.000028,0.000028,0.000516,0.000028,0.000028,0.000272,0.000028,0.000028,0.000028,...,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028
9,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000311,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032


<b>Step 5:</b> We do the same thing again, but focused on the documents and topics (rather than words & topics) to generate the <b>theta</b> document-topic matrix. Here, <b>alpha</b> is used as the smoothing value.

In [10]:
theta_df = df.groupby(['#doc', 'topic'])['topic'].count().reset_index(name ='topic_count')

theta_df[:10]

theta = pivot_and_smooth(theta_df, alpha, '#doc', 'topic', 'topic_count')

theta[:10]

ValueError: operands could not be broadcast together with shapes (138,45) (9,) 

The above processing steps represent about 90% of the work needed to prepare the mallet data for visualization with the LDAvis library. Now that all of the data has been formatted for pyLDAvis, we can queue it up and pass it to the visualization library.

In [ ]:
import pyLDAvis

data = {'topic_term_dists': phi,
        'doc_topic_dists': theta,
        'doc_lengths': list(docs['doc_length']),
        'vocab': list(vocab['type']),
        'term_frequency': list(vocab['term_freq'])
}

vis_data = pyLDAvis.prepare(**data)

pyLDAvis.display(vis_data)